# Попробуем произвести автоматическое извлечение предложений, в которых присутствует конгламерация слов библейского корпуса, частотность которых > 5. Для этого создадим корпус слов, в котором библейские слова слова будут обозначаться таргетом 1, а слова биологического корпуса таргетом 0. При этом, чтобы доля библейских слов в тестовых предложениях была ближе к реальной, в обучающий датасет мы решили включить и стоп-слова, присваивая им таргет 0 (т.е. слово не библейское).






In [ ]:
import pandas as pd
import numpy as np

In [ ]:
bio = pd.read_csv('/content/prep_BIO_SW.csv').dropna()
bio

,Unnamed: 0,text,cleaned_text,removed_terms_text,len
0,0,bs_bs_banner\n\nINVITED REVIEW SERIES:\nSTEM C...,bs bs banner invited review series stem cells ...,bs bs banner invited review series stem cells ...,140
1,1,IVAN BERTONCELLO AND JONATHAN L. MCQUALTER\n\n...,ivan bertoncello and jonathan mcqualter lung h...,and jonathan health research centre department...,494
2,2,"In this\nreview, we describe the attributes of...",in this review we describe the attributes of a...,in this review we describe the of adult stem a...,191
3,3,We describe the power and\nlimitations of expe...,we describe the power and limitations of exper...,we describe the power and of strategies and us...,214
4,4,The review summarizes\nrecent progress and obs...,the review summarizes recent progress and obst...,the review summarizes recent progress and in d...,161
...,...,...,...,...,...
120439,121158,45.,,,1
120440,121159,"Sheth RD, Jin M, Bhut BV, et al.",sheth rd jin m bhut bv et al,rd bv et al,29
120441,121160,(2014) Affinity precipitation of a monoclonal ...,affinity precipitation of a monoclonal antibo...,affinity precipitation of a from an harvest ...,131
120443,121162,"© 2015 Rachel Chen, et al., licensee AIMS Press.",rachel chen et al licensee aims press,chen et al aims,39


In [ ]:
bio.drop(['Unnamed: 0'], inplace=True, axis=1)

In [ ]:
bio.rename({"removed_terms_text":"prep_text"}, axis=1, inplace=True) # переименуем колонку

In [ ]:
bible = pd.read_csv("/content/prep_asb_SW.csv")
bible.prep_text = bible.prep_text.astype(str)
bible

,Unnamed: 0,Text,cleaned_text,prep_text,len
0,0,In the beginning God created the heavens and t...,in the beginning god created the heavens and t...,in the beginning god created the heavens and t...,55
1,1,And the earth was waste and void; and darkness...,and the earth was waste and void and darkness...,and the earth was waste and void and darkness...,134
2,2,"And God said, Let there be light: and there wa...",and god said let there be light and there wa...,and god said let there be light and there wa...,54
3,3,"And God saw the light, that it was good: and G...",and god saw the light that it was good and g...,and god saw the light that it was good and g...,85
4,4,"And God called the light Day, and the darkness...",and god called the light day and the darkness...,and god called the light day and the darkness...,63
...,...,...,...,...,...
35345,35345,and if any man shall take away from the words ...,and if any man shall take away from the words ...,and if any man shall take away from the words ...,185
35346,35346,"He who testifieth these things saith, Yea: I c...",he who testifieth these things saith yea i c...,he who testifieth these things saith yea i c...,58
35347,35347,"Amen: come, Lord Jesus.",amen come lord jesus,amen come lord jesus,23
35348,35348,The grace of the Lord Jesus be with the saints.,the grace of the lord jesus be with the saints,the grace of the lord jesus be with the saints,47


In [ ]:
bible.drop(['Unnamed: 0'], inplace=True, axis=1)

In [ ]:
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stops.add('') # можем добавить одно слово
# для нескольких пишем цикл

In [ ]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
def create_word_list(df, threshold=5): # формируем словарь: не берем слова, которые встречаются меньше 5 раз
    words_dict = {}
    for prep_text in df.prep_text.values:
        for word in prep_text.split(' '): # для слов в каждом предложении
            if len(word) <= 2 or word in stops: # не берем стоп-слова и "слова", состоящие из 2 букв
                continue
            if word in words_dict.keys(): # массив ключей будет пустой
                words_dict[word] += 1
            else:
                words_dict[word] = 1 # если ворд не включах, мы его не встречали, то мы его тоже добавляем в словарь
    words = [] # те слова, которые мы возвращаем
    for word, c in words_dict.items(): # метод словаря: если с больше, чем 5, то мы добавляем его в наш массив слов
        if c > threshold:
            words.append(word)
    return words


In [ ]:
# Получили списки слов из библейского и экономического корпусов
bio_words = create_word_list(bio)
bible_words = create_word_list(bible)

In [ ]:
len(bio_words), len(bible_words)

(6884, 4749)

In [ ]:
bible_words

['beginning',
 'god',
 'created',
 'heavens',
 'earth',
 'waste',
 'void',
 'darkness',
 'upon',
 'face',
 'deep',
 'spirit',
 'moved',
 'waters',
 'said',
 'let',
 'light',
 'saw',
 'good',
 'divided',
 'called',
 'day',
 'night',
 'evening',
 'morning',
 'one',
 'firmament',
 'midst',
 'divide',
 'made',
 'heaven',
 'second',
 'gathered',
 'together',
 'unto',
 'place',
 'dry',
 'land',
 'appear',
 'gathering',
 'seas',
 'put',
 'forth',
 'grass',
 'herbs',
 'yielding',
 'seed',
 'fruit',
 'trees',
 'bearing',
 'kind',
 'wherein',
 'thereof',
 'brought',
 'third',
 'lights',
 'signs',
 'seasons',
 'days',
 'years',
 'give',
 'two',
 'great',
 'greater',
 'rule',
 'stars',
 'also',
 'set',
 'fourth',
 'swarms',
 'living',
 'creatures',
 'birds',
 'fly',
 'open',
 'sea',
 'every',
 'creature',
 'moveth',
 'wherewith',
 'winged',
 'bird',
 'blessed',
 'saying',
 'fruitful',
 'multiply',
 'fill',
 'fifth',
 'bring',
 'cattle',
 'creeping',
 'things',
 'beasts',
 'everything',
 'creepeth'

## Формируем корпус слов, в котором библейские представлены с таргетом 1, а стоп-слова и слова биологического корпуса с таргетом 0.

In [ ]:
words = []
for word in stops: # сначала все стоп-слова
    words.append(word)
for word in bio_words:
    words.append(word)
for word in bible_words:
    words.append(word)
len(words)

11812

In [ ]:
corpus = pd.DataFrame({"word": words, "target": [0] * (len(bio_words) + len(stops)) + [1] * len(bible_words)})
corpus

,word,target
0,your,0
1,most,0
2,isn't,0
3,hadn,0
4,i,0
...,...,...
11807,mediator,1
11808,ensample,1
11809,laodicea,1
11810,overcometh,1


In [ ]:
corpus[corpus.target == 1]

,word,target
7063,beginning,1
7064,god,1
7065,created,1
7066,heavens,1
7067,earth,1
...,...,...
11807,mediator,1
11808,ensample,1
11809,laodicea,1
11810,overcometh,1


# Обучение модели

In [ ]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device available for running: ")
print(device)

Device available for running: 
cuda


In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel, AdamW

model_checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_checkpoint, padding=True, truncation=True)
# model_tf = DistilBertModel.from_pretrained(model_checkpoint, num_labels=2).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## В качестве модели использовали предобученный "distilbert"
(https://colab.research.google.com/github/DhavalTaunk08/Transformers_scripts/blob/master/Transformers_multilabel_distilbert.ipynb)

Что дает добавление drop out и dense layer=линеар

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained(model_checkpoint)# стандартный Берт
        self.pre_classifier = torch.nn.Linear(768, 768) # то, как устроена архитектура Берта - размерность вектора # видимо, чтобы стабилизировать обучение
        self.dropout = torch.nn.Dropout(0.1) # слой регуляризации - в нашем случае 10% весов зануляет
        self.classifier = torch.nn.Linear(768, 2) # стандартный размер выхода берта, мы превратили его в размерность 2 - изменили мультилэйбл # то, что выходную размерность Берта превращает в нужную нам: линейный

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask) # для каждого батча
        hidden_state = output_1[0] # выход берта - массив логитов
        pooler = hidden_state[:, 0] # в пуллере хранятся логиты
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)# функция ативации
        pooler = self.dropout(pooler)
        output = self.classifier(pooler) # еще один линейный слой
        return output

In [ ]:
model = DistilBERTClass().to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split

X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(corpus['word'],
                                                                corpus['target'],
                                                                test_size=0.2,
                                                                shuffle=True)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_encodings = tokenizer(list(X_train_tf.values), padding=True)
test_encodings = tokenizer(list(X_test_tf.values), padding=True)

In [ ]:
train_labels = torch.tensor(y_train_tf.values)
test_labels = torch.tensor(y_test_tf.values)


train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              train_labels)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             test_labels)

In [ ]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from tqdm import tqdm
from torch import nn
from torch.optim import Adam, AdamW

Посмотреть с tqdm

In [ ]:
optimizer_tf = AdamW(model.parameters(), lr=2e-5)

epochs = 10
criterion = nn.CrossEntropyLoss()
for epoch in tqdm(range(epochs)):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        optimizer_tf.zero_grad()
        texts, atts, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        output = model(texts, attention_mask=atts)

        loss = criterion(output, labels)
        loss.backward()
        epoch_loss += loss

        optimizer_tf.step()

100%|██████████| 10/10 [04:01<00:00, 24.18s/it]


In [ ]:
predictions = []
probabilities = []
original_labels = []

with torch.no_grad():
    model.eval()
    for batch in test_loader:
        texts, atts, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        output = model(texts, attention_mask=atts)
        probabilities.extend(output)
        predictions.extend(output.argmax(axis=1).cpu())
        original_labels.extend(labels.cpu())

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(original_labels, predictions))

              precision    recall  f1-score   support

           0       0.80      0.82      0.81      1386
           1       0.74      0.71      0.72       977

    accuracy                           0.78      2363
   macro avg       0.77      0.76      0.77      2363
weighted avg       0.77      0.78      0.77      2363



In [ ]:
torch.save(model.state_dict(), '/content/with_SW_word_clf.pth')


# Смотрим, как модель определяет библейские цитаты

In [ ]:
import pandas as pd

In [ ]:
model = DistilBERTClass()
#model.load_state_dict(torch.load("/content/with_SW_word_clf.pth"))
model.eval()

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [ ]:
# witn CUDA
model = DistilBERTClass().to(device)
model.load_state_dict(torch.load("/content/with_SW_word_clf.pth"))

<All keys matched successfully>

In [ ]:
economist = pd.read_csv("/content/prep_bio_Economist_SW.csv")
economist

,Unnamed: 0,Text,cleaned_text,len
0,0,FOR THE past four billion years or so the only...,for the past four billion years or so the only...,154
1,1,Sometimes the gene would be damaged or scrambl...,sometimes the gene would be damaged or scrambl...,96
2,2,From that raw material arose the glories of na...,from that raw material arose the glories of na...,62
3,3,"But beneath it all, gene begat gene.That is no...",but beneath it all gene begat gene that is no ...,58
4,4,Now genes can be written from scratch and edit...,now genes can be written from scratch and edit...,90
...,...,...,...,...
35847,35972,"He found that, after 15 days, parasite loads i...",he found that after days parasite loads in ani...,269
35848,35973,Dr Kumar plans to try—though he will use genet...,dr kumar plans to try though he will use genet...,137
35849,35974,"But many, probably most, doctors are suspiciou...",but many probably most doctors are suspicious ...,165
35850,35975,"Dr Kumar’s findings are, nevertheless, interes...",dr kumar s findings are nevertheless interesting,49


Еще раз пройтись по предобработке текста!!!

In [ ]:
# В процессе работы обнаружили, что требуется дополнительная предобработка текстов статей The Economist
def preprocess_economist(prep_economist):
    prep_economist.loc[:, 'cleaned_text'] = prep_economist['cleaned_text'].astype(str)
    prep_economist['cleaned_text'] = prep_economist['cleaned_text'].replace(' $', '', regex=True) # все пробелы в самом конце строки (они лишние) и мешали адекватно разделить на слова
    prep_economist['cleaned_text'] = prep_economist['cleaned_text'].replace('^ ', '', regex=True) # нчало строки - если она начинается с пробела, а не со слова, мы его убираем
    prep_economist["number_words"] = prep_economist['cleaned_text'].apply(lambda x: len(x.split(' '))) # считаем кол-во слов в предложении
    prep_economist = prep_economist[prep_economist.number_words >= 2] # убираем предложения из 2х или 1 слова
    prep_economist.drop('number_words', axis=1) # удаляем вспомогат колонку
    return prep_economist

In [ ]:
prep_economist = preprocess_economist(economist)

In [ ]:
prep_economist.cleaned_text.iloc[1]

'sometimes the gene would be damaged or scrambled the copying imperfect or undertaken repeatedly'

In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
# собираем все слова из Экономиста
words = []
for text in prep_economist["cleaned_text"].values:
    sent = text.split(' ')
    len_sent = len(sent)
    for i in range(len_sent):
        words.append(sent[i])

Нам нужно определить, какие слова являются началом предложения, а какие - концом, для того, чтобы считать количество библейских слов в рамках именно одного предложения.
bos - beginning of the sequence
eos - end of the sequence

In [ ]:
bos_eos = [] #массив длиной len(words), состоящий из 0, 1, 2:
# 0 -- текущее слово -- не начало и не конец какого-либо предложения
# 1 -- текущее слово -- начало предложения
# 2 -- текущее слово -- конец предложения
#words:    [this, is, a, sequence, this, is, a, second, sentence]
#bos_eos:  [1,    0,  0,     2   ,  1     0, 0,    0,       2]
#preds:    [1,    0,  0,   0,        1...] #1 -- библ, 0 -- биоло

for sent in prep_economist["cleaned_text"].values:
    bos_eos.append(1) # добавить токенов столько, сколько у нас слов в предложении
    len_sent = len(sent.split(' '))
    for i in range(1, len_sent - 1): # мы нулей добавили количеством оставшихся от вычета первого и последнего "слова"
        bos_eos.append(0)
    bos_eos.append(2)

In [ ]:
len(words), len(bos_eos) # сколько слов и сколько "лейблы"

(868117, 868117)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
infer_encodings = tokenizer(words, padding=True)
infer_dataset = TensorDataset(torch.tensor(infer_encodings['input_ids']),
                              torch.tensor(infer_encodings['attention_mask']))

In [ ]:
infer_loader = DataLoader(infer_dataset, batch_size=256, shuffle=False)

In [ ]:
device = torch.device("cuda") # перекидываем модель на куду
model = model.to(device)

In [ ]:
predictions = []
probabilities = []

with torch.no_grad():
    model.eval()
    for batch in tqdm(infer_loader):
        word, atts = batch[0].to(device), batch[1].to(device)
        output = model(word, attention_mask=atts)
        probabilities.extend(output)
        predictions.extend(output.argmax(axis=1))


100%|██████████| 3392/3392 [03:26<00:00, 16.41it/s]


In [ ]:
preds = np.array(predictions)
words = np.array(words)

In [ ]:
# слова, которая модель посчитала библейскими:
for word in words[preds == 1]:
    print(word)

Выходные данные были обрезаны до нескольких последних строк (5000).
catalogue
chemicals
viruses
metabolisms
everybody
counts
truly
living
beings
making
bacterium
least
genome
bacterium
would
one
last
nails
coffin
vitalism
curiously
persistent
idea
something
biology
mere
physics
chemistry
curiosity
drives
project
though
dr
venter
believes
cell
built
around
minimum
genome
necessary
life
would
important
tool
biotechnology
stripped
cell
could
extra
biochemical
pathways
engineered
make
useful
chemical
products
drugs
fuels
without
fear
cell
resources
would
diverted
useless
tasks
finding
extra
biochemical
pathways
also
agenda
another
dr
venter
characteristics
ability
mix
business
pleasure
end
fitted
racing
yacht
sorcerer
ii
laboratory
sailing
round
world
collecting
samples
flown
back
rockville
undergo
whole
ecosystem
genome
analysis
process
tries
extract
analyse
dna
bacterial
species
sample
since
bacteria
abundant
form
life
earth
hopes
process
eventually
reveal
genes
planet
dr
venter
first
ca

In [ ]:
len(words[preds == 1])  # words_e

479469

## Далее проходимся по предложениям текстов корпуса The Economist, чтобы выявить те, где встречаются n-gramms, состоящие из слов, которые модель посчитала "библейскими".

In [ ]:
# predictions - nmpy массив
# пара библейских слов одно за другим
for i in range(1, len(preds)):
    if preds[i - 1] == 1 and preds[i] == 1:
        print(words[i - 1], words[i])

Выходные данные были обрезаны до нескольких последних строк (5000).
regulate aspects
immune system
particular low
low status
status individuals
individuals showed
showed high
high levels
genes associated
various immune
immune related
related cells
chemical signalling
signalling factors
general immune
immune response
involves tissue
tissue swelling
increased immune
immune cell
cell activity
affected area
area although
explicitly examine
simian charges
charges chronic
chronic generalised
generalised inflammation
risk factor
long list
ailments ranging
heart trouble
disease finally
team investigated
mechanisms behind
gene expression
previous work
low rank
rank individuals
individuals showed
showed different
different levels
hormones called
called glucocorticoids
regulate immune
immune system
system activity
also found
found changes
cells within
animals immune
immune system
first time
time evidence
phenomenon called
called epigenetic
epigenetic change
work epigenetics
epigenetics currently


Этот подход нам ничего не дал.

In [ ]:
# три библейских слова одно за другим
for i in range(2, len(preds)):
    if preds[i - 2] == 1 and preds[i - 1] == 1 and preds[i] == 1:
        print(words[i - 2], words[i - 1], words[i])

Выходные данные были обрезаны до нескольких последних строк (5000).
killed five people
american people dr
people dr wessely
dr wessely distinguishes
wessely distinguishes acute
distinguishes acute episodes
often occur among
occur among schoolchildren
investigated rapidly symptoms
rapidly symptoms vanish
symptoms vanish within
vanish within days
become chronic patients
chronic patients beliefs
media shelkovsk may
chechen incident turns
people find hard
believe says dr
says dr jakab
anxiety related illness
closely mimic illness
mimic illness caused
canister containing highly
containing highly radioactive
highly radioactive caesium
radioactive caesium chloride
goi nia brazil
junkyard owner exposing
contaminated yet another
yet another people
another people reported
people reported vomiting
reported vomiting diarrhoea
acute radiation sickness
radiation sickness dr
sickness dr jakab
dr jakab says
jakab says medical
says medical staff
medical staff need
world health organisation
speed patien

In [ ]:
# четыре библейских слова одно за другим - просто выводии совосочетания
for i in range(3, len(preds)):
    if preds[i - 3] == 1 and preds[i - 2] == 1 and preds[i - 1] == 1 and preds[i] == 1:
        print(words[i - 3], words[i - 2], words[i - 1], words[i])

Выходные данные были обрезаны до нескольких последних строк (5000).
walked south single file
cable bridges grey blue
bridges grey blue snow
grey blue snow melt
blue snow melt gushing
snow melt gushing beneath
melt gushing beneath us
legs like trousers skulked
like trousers skulked among
trail days without roads
swedish touring association proposed
north south walking route
nervous thrown together like
wild flowers avens gentian
flowers avens gentian meadowsweet
provisions pasta coffee bricks
treacle dark rye bread
dark rye bread slabs
glaciers lying around like
lying around like toys
mi shelters turf mounds
mi place names pimped
k totj kka massif
totj kka massif ahead
c akcahj lm n
willow saxifrage speedwell arctic
saxifrage speedwell arctic bellflower
speedwell arctic bellflower mica
arctic bellflower mica specks
bellflower mica specks twinkled
bj rn borg underpants
tie dyed face hovering
long broad valley called
broad valley called tj
valley called tj ktjavagge
called tj ktjavagge ma

в стоп-слова можно включить high, new, others, make, recently, necessary, really, etc.

Нам нужно определить, являются началом предложения, а какие - концом, для того, чтобы считать количество библейских слов в рамках именно одного предложения.
bos - beginning of the sequence
eos - end ...

In [ ]:
bos_eos = [] #массив длиной len(words), состоящий из 0, 1, 2:
# 0 -- текущее слово -- не начало и не конец какого-либо предложения
# 1 -- текущее слово -- начало предложения
# 2 -- текущее слово -- конец предложения
#words:    [this, is, a, sequence, this, is, a, second, sentence]
#bos_eos:  [1,    0,  0,     2   ,  1     0, 0,    0,       2]
#preds:    [1,    0,  0,   0,        1...] #1 -- библ, 0 -- биоло

for sent in prep_economist["cleaned_text"].values:
    bos_eos.append(1) # добавить токенов столько, сколько у нас слов в предложении
    len_sent = len(sent.split(' '))
    for i in range(1, len_sent - 1): # мы нулей добавили количеством оставшихся от вычета первого и последнего "слова"
        bos_eos.append(0)
    bos_eos.append(2)

In [ ]:
len(words), len(bos_eos) # сколько слов и сколько "лейблов"

(868117, 868117)

In [ ]:
sent_preds = [] #массив массивов, где каждый массив -- это предсказания для одного предложения
#bos_eos =    [1, 0, 0, 2,   1, 0, 2]
#preds =      [0, 1, 1, 1,   0, 0, 1]
#sent_preds = [[0, 1, 1, 1], [0, 0, 1]]
i = 0 # текущий индекс слова/предсказания
while i < len(preds):
    sent = [preds[i]]
    i += 1 # добавляем текущее предсказание и дем к след предсказанию
    # в след цикле добавляем предсказание для всех слов, кот не являются ни началом, ни концом
    while i < len(preds) and bos_eos[i] != 2: # пока мы не достигли конца текущего предложения, мы доваляем текущее предсказание (кот не являются концом предложения)
        sent.append(preds[i])
        i += 1
    sent.append(preds[i]) # для последнего слова предложения
    sent_preds.append(np.array(sent)) #предсказание текущего предложения и переходим к след
    i += 1

In [ ]:
sent_preds[0] # первое предложение

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

In [ ]:
# Здесь мы смотрим
for i, sent in enumerate(sent_preds):
    if sent.mean() > 0.6: # доля единиц в нашем предложении - кол-во единиц разделили на сумму кол-ва единиц и нулей
        #выводим все слова в текущем предложении sent, которая модель посчитала библейскими:
        # print(np.array(prep_economist.iloc[i].cleaned_text.split(' '))[np.array(sent) == 1])
        print(prep_economist.iloc[i].Text)

It touches everything.
Kenneth Cukier hosts.
WHENCE morality?
In every country, women like rich men, and men like young attractive women.
Kenneth Cukier hosts.
Fourth Estate; 310 pages; £20.
*6 Finding living planets
Living hosts do.
COWBOY or spaceman?
PADDINGTON loved marmalade.
Spotted owls get money.
Kenneth Cukier hosts.
Alok Jha hosts.
Tom Standage hosts.
Voters get transparency.
Businesses get certainty.
Kenneth Cukier hosts.
Sturdier armour helped.
Nothing scary.
Meddling bureaucrats.
Giant schools.
The clever sleuthing worked.
■Dig deeper:
Some sardonically urged the brewer to “please stop killing innocent people”.
In parts of Asia and east Africa, swarms of locusts have stripped fields.
Beavers wear implants.
Elephants, proverbially, never forget.
Others dispute this.
Grief became struggle.
Learn from others.
Advertising ArchivesGRUB, filth, grime, muck, gunk, slag, grit, grunge, smut, dross, dust, sludge, squalor.
Insulted, hounded and despised, dirt these days has nowhere t

In [ ]:
# Здесь мы вызываем предложения, в которых более 8 библейских единиц
for i, sent in enumerate(sent_preds):
    if sent.mean() > 0.6 and sent.sum() >= 8:

        print(prep_economist.iloc[i].Text)

In [ ]:
# смотрим подряд идущие "библейские" единицы
for i, sent in enumerate(sent_preds):
    number_ones = 0 #количество подряд идущих единиц
    for pred in sent:
        if pred == 1:
            number_ones += 1
            if number_ones > 5:
                #выводим все слова в текущем предложении sent, которая модель посчитала библейскими
                print(np.array(prep_economist.iloc[i].cleaned_text.split(' '))[np.array(sent) == 1])
                break
        if pred == 0:
            number_ones = 0


Выходные данные были обрезаны до нескольких последних строк (5000).
['rangers' 'park' 'guards' 'even' 'well' 'informed' 'tourists' 'could'
 'use' 'record' 'handy' 'data' 'work' 'play']
['gamba' 'protected' 'area' 'complex' 'coast' 'gabon' 'patrolling'
 'rangers' 'make' 'daily' 'logs' 'deposits' 'flotsam' 'industrial' 'world'
 'arrived' 'remote' 'atlantic' 'beaches' 'time' 'record' 'comings'
 'goings' 'local' 'leatherback' 'turtles']
['aware' 'study' 'required' 'broad' 'international' 'scientific'
 'collaboration' 'humboldt' 'became' 'early' 'champion']
['humboldt' 'drive' 'establish' 'network' 'meteorological' 'geomagnetic'
 'measurement' 'stations' 'across' 'northern' 'asia' 'later' 'hailed'
 'encyclopaedia' 'britannica' 'scientific' 'conspiracy' 'nations' 'one'
 'noblest' 'fruits' 'modern' 'civilisation' 'although' 'humboldt'
 'possibly' 'admired' 'man' 'europe' 'napoleon' 'bonaparte' 'allegedly'
 'envied' 'fame' 'star' 'faded' 'end' 'th' 'century']
['oxygen' 'level' 'fell' 'though' 